In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:

# chromedriver = r"C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe" # path to the chromedriver executable
# os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
pages_url=[]

prices,areas,direction,bedroom,bathroom,living_room,street_width,House_age,views=[], [], [], [], [], [], [], [], []

In [4]:
for i in range(2,350):
    url ="https://sa.aqar.fm/%D9%81%D9%84%D9%84-%D9%84%D9%84%D8%A8%D9%8A%D8%B9/%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/%D8%B4%D9%85%D8%A7%D9%84-%D8%A7%D9%84%D8%B1%D9%8A%D8%A7%D8%B6/2"+str(i)
    response=requests.get(url)
    page=response.text
    soup=BeautifulSoup(page, "html")
    for el in soup.find_all('a', {'class':"listTitle"}):
        pages_url.append(el["href"])

In [5]:
# for el in soup.find_all('a', {'class':"listTitle"}):
#     pages_url.append(el["href"])

In [6]:
# soup.find_all('a', {'class':"listTitle"})

In [7]:

def collect_aqar_data():
   
   for n in pages_url:
        response=requests.get('https://sa.aqar.fm'+n)
        page=response.text
        soup=BeautifulSoup(page, "html.parser")
        if soup.find("span", class_="listingPagePrice") is None :
            prices.append(np.nan)
        else:
            prices.append(soup.find("span", class_="listingPagePrice").find("span").findNext().text) 
#-------------------------------------------------------
        if soup.find(text="المساحة") is None:
            areas.append(np.nan)
        else:
            areas.append(soup.find(text="المساحة").find_previous().find_previous().find_previous().text)
#-------------------------------------------------------       
        if soup.find(text="الواجهة") is None:
            direction.append(np.nan)
        else:
            direction.append(soup.find(text="الواجهة").find_previous().find_previous().find_previous().text)
#-------------------------------------------------------            
        if soup.find(text="غرف النوم") is None:
            bedroom.append(np.nan)
        else:
            bedroom.append(soup.find(text="غرف النوم").find_previous().find_previous().find_previous().text)    
#-------------------------------------------------------        
        if soup.find(text="دورات مياه") is None:
            bathroom.append(np.nan)
        else:
            bathroom.append(soup.find(text="دورات مياه").find_previous().find_previous().find_previous().text)
#-------------------------------------------------------        
        if soup.find(text="الصالات") is None :
            living_room.append(np.nan)
        else:
            living_room.append(soup.find(text="الصالات").find_previous().find_previous().find_previous().text)
#-------------------------------------------------------        
        if soup.find(text="عرض الشارع") is None:
            street_width.append(np.nan)
        else:
            street_width.append(soup.find(text="عرض الشارع").find_previous().find_previous().find_previous().text)
#-------------------------------------------------------        
        if soup.find(text="عمر العقار") is None:
            House_age.append(np.nan)
        else:
            House_age.append(soup.find(text="عمر العقار").find_previous().find_previous().find_previous().text)
#-------------------------------------------------------       
        if soup.find("table", class_="dc-listingInfoTable") is None:
            views.append(np.nan)
        else:
            views.append(soup.find("table", class_="dc-listingInfoTable").find("span").text) 

In [8]:
collect_aqar_data()

In [9]:
df=pd.DataFrame(list(zip(prices,areas,direction,bedroom,bathroom,living_room,street_width,House_age,views)),
                      columns=["price","area","direction","bedroom","bathroom","living_room","street_width","house_age","views"])

In [10]:
from bs4 import BeautifulSoup
import requests
import csv
from itertools import zip_longest
import math
import pandas as pd
import patsy
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from sklearn import metrics

import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression,Lasso, Ridge, ElasticNet,LassoCV, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [11]:
import csv
file=[prices,areas,direction,bedroom,bathroom,living_room,street_width,House_age,views]
a=zip_longest(*file)
with open(r"\Users\HP\Desktop\aqar\aqar_1.csv","w") as myfile:
    wr=csv.writer(myfile)
    wr.writerow(["price","area","direction","bedroom","bathroom","living_room","street_width","house_age","views"])


In [12]:
df

,price,area,direction,bedroom,bathroom,living_room,street_width,house_age,views
0,"5,500,000",540 م²,غرب,4,5,2,15 م,جديد,183
1,"3,700,000",370 م²,شمال,5,5,2,15 م,جديد,48
2,"2,250,000",300 م²,شرق,4,4,1,15 م,NaN,43
3,"3,000,000",337 م²,شمال,5,5,2,20 م,جديد,53
4,"2,500,000",312 م²,جنوب غربي,4,3,5,20 م,5 سنة,79
...,...,...,...,...,...,...,...,...,...
6955,"2,600,000",600 م²,جنوب,5,5,2,14 م,22 سنة,6119
6956,"2,000,000",375 م²,غرب,4,5,4,15 م,14 سنة,5967
6957,"4,500,000",600 م²,شمال,5,5,4,15 م,جديد,6935
6958,"4,800,000",800 م²,جنوب,7,5,2,15 م,12 سنة,3861


In [13]:
df.to_csv("aqar_1.csv")